# Insights proposal
 
## Insight 1:

### Target:

We are exploring the relationship between player position and their physical conditions

### Methodology:

- We have chosen heights without shoes, weights, and body fat percentage as features to represent the physical trials of the player 

- We calculate the mean values on these attributes for each position group 
- We use bar chart to show the values in each group for better comparison

### Result: (on average)
1. for height: PG < SG < SF < PF < C
2. for weight: PG < SG < SF < PF < C
3. for body fat percentage: PG < SG < SF < PF < C

### Interpretation:

PG stands for point guards, whose primary role is to carry the ball across the perimeter, hence players in this position are on average shorter and lighter

On the contrary, C (centers) are mainly responsible for blocking and defending, hence players in center position on average would be taller and heavier 


## Insight 2:

### Target:

We are exploring the relationship between player position and their athletic performace

### Methodology:

- We have chosen max_vertical_leap, lane_agility_time, bench_press as features to represent the athletic performance of the player, as these features can be considered as proxy values for explosive power, speed and endurance

- We calculate the mean values on these attributes for each position group 
- We use spider charts to see the performance in each dimension for each position 

### Result: (on average)
1. C and PF: perform the best in bench press
2. PG and SG: perform the best in lane agibility 
3. SG and PG: perform the best in vertical leap

### Interpretation:

Shooting Guards and Point Guards are responsible for shooting and dunking, hence they normally have the best jumping skills and explosive power 

Point Guards and  Shooting Guards require high agility to move pass the perimeter, hence they have the best speed and are the quickest players on the court  


Center and Power Forwards require players are the most physically demanding positions, hence we can see players in these two positsion in general have greater endurance 

## Insight 3:

### Target:

We aim to analyse the distribution of NBA draft picks across different sources (college, high school, and others) to understand the primary recruitment channels for NBA teams.

### Methodology:

- Data Categorisation: We categorise the source of each NBA draft pick into three groups: College, High School, and Other.
- Visualisation: A bar chart is created to display the count of draft picks from each source, providing a clear view of the dominant recruitment channels.

### Result:
1. College recruits make up the majority of NBA draft picks.
2. High School recruits have a relatively small representation compared to college recruits.
3. Other sources provide some NBA talent but are less common than college pathways.

### Interpretation:

The preference for players with collegiate experience reflects the NBA’s reliance on college programs as a primary development ground for talent. This preference suggests that collegiate leagues provide a structured training environment that helps players build essential skills for the NBA.


## Insight 4:

### Target:

We explore the stability of NBA franchises by assessing the duration that each city has hosted an NBA team, indicating strong or transient NBA markets.

### Methodology:

- Data Cleaning: The dataset is cleaned to merge similar city names (e.g., combining "Minneapolis" and "Minnesota") and handle missing values by filling current years for active franchises.
- Duration Calculation: We calculate the number of years each city has hosted a franchise and aggregate these durations for comparison.
- Visualisation: A horizontal bar chart is used to display the total years of NBA presence for each city, sorted from longest to shortest duration.

### Result:

1. Los Angeles, New York, and Boston are among the cities with the longest NBA presence, indicating high franchise stability.
2. Cities with shorter hosting durations may represent newer franchises or locations with more team movement.

### Interpretation:

Cities with long-standing NBA franchises, such as Los Angeles and New York, reflect established markets that benefit from consistent fan engagement and revenue. The stability in these cities points to strong local support and significant market value, while newer or shorter-duration cities may still be developing their fan base and market presence.

In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / 'data' / 'nba.sqlite'

In [ ]:
# Set the maximum columns displayed to the number of columns in your DataFrame
pd.set_option('display.max_rows', None)


## Load data

In [ ]:
con = sql.connect(DATA_PATH)  # connect to the database

query = """
    SELECT *
    FROM draft_combine_stats
""" 

df = pd.read_sql(query, con)


## Feature selection

Proxy features for player's physical condition:
- height_wo_shoes
- weight 
- body_fat_pct

Proxy features for player's athletic performance: 
- max_vertical_leap
- lane_agility_time
- bench_press

In [ ]:
df = df[['position', 'height_wo_shoes', 'body_fat_pct', 'weight', 'max_vertical_leap', 'lane_agility_time', 'bench_press']]

## Quick data reporting

1. missing values
2. data types

In [ ]:
#report on missing values
print(df.isnull().sum())

#report on data type
print(df.dtypes)

## Data wrangling for draft_combine_stats (with selected columns)

1. drop rows if target variable 'position' is empty 
2. convert object to float when necessary (for 'weight' column in specific)
3. handle missing values in attribute columns (fill with mean)
4. clean up / standardised labels (for 'position' column in specific)
  


In [ ]:
def drop_empty_str_row(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    drop rows in the dataframe if a specified column is an empty string ''.

    Arg:
    df (pd.DataFrame): input dataframe
    col (str): input column name 

    Returns:
    pd.DataFrame: dataframe after the clean-up
    """
    return df[df[col] != '']

In [ ]:
df_clean = drop_empty_str_row(df, "position")
print('number of filtered rows:', df.shape[0] - df_clean.shape[0])

In [ ]:
def convert_float(df: pd.DataFrame, col_list: list) -> pd.DataFrame:
    """
    replace empty strings or None in the specified column with NaN 
    and converts the column to float

    Arg:
    df (pd.DataFrame): input dataframe
    col_list (list): a list of column names to convert to float 

    Returns:
    pd.DataFrame: dataframe after the clean-up
    """
    import numpy as np
    for col in col_list:
        df[col] = df[col].replace(['', None], np.nan)
        df[col] = df[col].astype(float)

    return df


In [ ]:
df_clean = convert_float(df_clean, ["weight", "body_fat_pct"])


In [ ]:
def fill_na_with_mean(df: pd.DataFrame, col_list: list) -> pd.DataFrame:
    """
    fill NaN values in the specified list of columns with the mean of each column.

    Arg:
    df (pd.DataFrame): input dataframe
    col_list (list): a list of column names to fill NaN values in

    Returns:
    pd.DataFrame: dataframe after the processing 
    """
    for col in col_list:
        mean_val = df[col].mean()
        df[col] = df[col].fillna(mean_val)
    return df

In [ ]:
df_clean = fill_na_with_mean(df_clean, ['height_wo_shoes', 
'body_fat_pct', 'weight', 'max_vertical_leap', 'lane_agility_time', 'bench_press'])

In [ ]:
def cleanup_position(df: pd.DataFrame) -> pd.DataFrame:
    """
    clean up hybrid position labels 
    
    Args:
    df (pd.DataFrame): input dataframe
   
    Returns:
    pd.DataFrame: dataframe after the clean-up
    """
    position_dict = {
        'C-PF': 'C-PF',
        'PF-C': 'C-PF',
        'PF-SF': 'PF-SF',
        'SF-PF': 'PF-SF',
        'SG-PG': 'PG-SG',
        'PG-SG': 'PG-SG',
        'SF-SG': 'SG-SF',
        'SG-SF': 'SG-SF'
    }
    df['position'] = df['position'].replace(position_dict)
    return df

In [ ]:
df_clean = cleanup_position(df_clean)

In [ ]:
df_clean['position'].unique()

In [ ]:
def groupby_mean(df: pd.DataFrame, groupby_col: str) -> pd.DataFrame:
    """
    aggregate the df by a specified column and calculate the mean for other columns

    Args:
    df (pd.DataFrame): input dataframe
    groupby_col (str): column to perform groupby action on 

    Returns:
    pd.DataFrame: dataframe after the groupby action
    """
    agg_df = df.groupby(groupby_col, as_index=False).mean()
    return agg_df

In [ ]:
df_agg = groupby_mean(df_clean, "position")
df_agg

## Visualisation
1. bar chart to see average for each position 
2. radar chart for each position

#### Insight 1: position VS body conditions

In [ ]:
def bar_plot_position(df: pd.DataFrame, metrics: list) -> None:
    """
    plot bar chart for each specified metric by position
    
    Args:
    df (pd.DataFrame): dataframe (after aggregation)
    metrics (list): list of metrics (average value)
    
    Returns:
    None
    """
    for metric in metrics:
        #sort by values 
        sorted_df = df.sort_values(by=metric, ascending=False)
        # plot bar chart
        plt.figure(figsize=(6, 2))
        plt.bar(sorted_df["position"], sorted_df[metric])
        plt.xlabel("Position")
        plt.title(f"{metric} by Position")
        plt.xticks(rotation=45)
        plt.show()


In [ ]:
bar_plot_position(df_agg, ['height_wo_shoes', 'body_fat_pct', 'weight'])

#### Insight 2: position VS athletic performance

In [ ]:
def rador_plot_position(df: pd.DataFrame) -> None:
    """
    create radar plots for each position
    
    Args:
    df (pd.DataFrame): dataframe containing 'max_vertical_leap', 'lane_agility_time', 'bench_press'
    
    Returns:
    None
    """
    #filter our hybrid position 
    df = df[df['position'].isin(['SG', 'PG', 'SF', 'PF', 'C'])]

    # normalised and transform data
    # max_vertical_leap: high means better explosive_power
    # lane_agility_time: shorter means faster speed -> take the inverse
    # bench_press: larger means greater endurance 
    min_leap = df['max_vertical_leap'].min()
    max_leap = df['max_vertical_leap'].max()
    min_agility = df['lane_agility_time'].min()
    max_agility = df['lane_agility_time'].max()
    min_press = df['bench_press'].min()
    max_press = df['bench_press'].max()
    df_normalized = df.copy()
    df_normalized['explosive_power'] = df['max_vertical_leap'].apply(lambda x: (x - min_leap) / (max_leap- min_leap))
    df_normalized['speed'] = df['lane_agility_time'].apply(lambda x: (1/x - 1/max_agility) / (1/min_agility - 1/max_agility))
    df_normalized['endurance'] = df['bench_press'].apply(lambda x: (x - min_press) / (max_press - min_press))

    # Create radar plots for each position
    for _, row in df_normalized.iterrows():
        # Values for the radar chart, add the first value to close the loop
        values = row[['explosive_power', 'speed', 'endurance']].values.tolist()
        values += values[:1]  # Close the loop

        # Define the angle for each axis
        angles = np.linspace(0, 2 * np.pi, 3, endpoint=False).tolist()
        angles += angles[:1]  # Close the loop

        # Initialize the radar chart
        fig = plt.figure(figsize=(4, 4))
        ax = fig.add_subplot(111, polar=True)

        # Draw the outline and fill in the area
        ax.fill(angles, values, color='skyblue', alpha=0.25)
        ax.plot(angles, values, color='blue', linewidth=2)

        # Set labels for each metric
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(['explosive_power', 'speed', 'endurance'], fontsize=10)

        # Set the title with the position name
        position_name = row['position']
        ax.set_title(f"{position_name} Profile", size=15, color='blue', y=1.1)

        # Show each radar plot individually
        plt.show()


In [ ]:
import numpy as np
rador_plot_position(df_agg)

#### Insight 3: NBA Draft Pick Distribution by Source

In [ ]:
def analyse_draft_picks():
    """
    This function loads draft pick data from a CSV file, categorizes draft picks by source
    (College, High School, Others), and visualises the distribution in a bar chart.
    """
    query = """
    SELECT *
    FROM draft_history
    """ 

    # Load the dataset
    draft_data = pd.read_sql(query, con)


    # Count the number of draft picks by organization type
    draft_counts = draft_data['organization_type'].value_counts()

    # Create categories for College, High School, and Others
    categories = {
        'College': draft_counts.get('College/University', 0),
        'High School': draft_counts.get('High School', 0),
        'Others': draft_counts.sum() - draft_counts.get('College/University', 0) - draft_counts.get('High School', 0)
    }

    # Convert the dictionary to a DataFrame and sort by the number of College draft picks
    categories_df = pd.DataFrame(list(categories.items()), columns=['Category', 'Number of Draft Picks'])
    categories_df = categories_df.sort_values(by='Number of Draft Picks', ascending=False)

    # Visualization: Bar chart showing the number of draft picks by category
    plt.figure(figsize=(8, 5))
    plt.bar(categories_df['Category'], categories_df['Number of Draft Picks'], color=['blue', 'green', 'red'])
    plt.xlabel("Category")
    plt.ylabel("Number of Draft Picks")
    plt.title("Number of Draft Picks by Category")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


# Call the function
analyse_draft_picks()


#### Insight 4: Stability of NBA Franchises

In [ ]:
def analyse_franchise_by_city():
    """
    This function loads draft pick data from a CSV file, categorizes draft picks by source
    (College, High School, Others), and visualises the distribution in a bar chart.
    """
    query = """
    SELECT *
    FROM team_history
    """
    from datetime import datetime

    # Load the dataset
    nba_data = pd.read_sql(query,con)

    # Display the first few rows to understand the structure (for testing)
    # print(nba_data.head())

    # Data Cleaning Adjustments

    # 1. Combine "Minneapolis" and "Minnesota" as "Minnesota"
    nba_data['city'] = nba_data['city'].replace({'Minneapolis': 'Minnesota', 'Minnesota': 'Minnesota'})

    # 2. Remove any blank or missing entries in the 'city' column
    nba_data = nba_data[nba_data['city'].notna() & (nba_data['city'] != '')]

    # 3. Merge "Kansas City-Omaha" and "Kansas City" into "Kansas City"
    nba_data['city'] = nba_data['city'].replace({'Kansas City-Omaha': 'Kansas City', 'Kansas City': 'Kansas City'})

    # 4. Remove "Capital" if it appears in the city names
    nba_data['city'] = nba_data['city'].str.replace('Capital', '', regex=False).str.strip()

    # Convert 'year_founded' and 'year_active_till' to datetime
    nba_data['year_founded'] = pd.to_datetime(nba_data['year_founded'], format='%Y')
    nba_data['year_active_till'] = pd.to_datetime(nba_data['year_active_till'], format='%Y', errors='coerce')

    # Fill any missing 'year_active_till' values with the current year (assuming these teams are still active)
    current_year = datetime.now().year
    nba_data['year_active_till'] = nba_data['year_active_till'].fillna(pd.Timestamp(current_year))

    # Calculate the number of years each team has been in its city
    nba_data['years_in_city'] = (nba_data['year_active_till'].dt.year - nba_data['year_founded'].dt.year).astype(int)

    # Aggregate years by city (regardless of franchise name) and sort by total years in descending order
    city_duration = nba_data.groupby('city')['years_in_city'].sum().reset_index()
    city_duration = city_duration.sort_values(by='years_in_city', ascending=False)

    # Visualization: Total years each city has hosted a franchise (sorted)
    plt.figure(figsize=(12, 8))
    plt.barh(city_duration['city'], city_duration['years_in_city'], color='skyblue')
    plt.xlabel('Total Years as Host City')
    plt.ylabel('City')
    plt.title('Total Years Each City Has Hosted an NBA Franchise (Sorted by Duration)')
    plt.gca().invert_yaxis()  # To have the longest-serving cities at the top
    plt.tight_layout()
    plt.show()

# Call the function
analyse_franchise_by_city()